In [1]:
import os
import json
import warnings
import traceback

import numpy as np
import pandas as pd

import grids

import dask
import dask.bag as db
from dask.diagnostics import ProgressBar

/short/en0/vhl548/miniconda3/envs/radar/lib/python3.7/site-packages/pyart/graph/cm.py:104: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if 'red' in spec:
/short/en0/vhl548/miniconda3/envs/radar/lib/python3.7/site-packages/pyart/graph/cm_colorblind.py:32: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if 'red' in spec:
/short/en0/vhl548/miniconda3/envs/radar/lib/python3.7/site-packages/heapdict.py:11: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  class heapdict(collections.MutableMapping):


In [2]:
def get_file(date):
    year = date.year
    date_str = date.strftime("%Y%m%d")
    time_str = date.strftime("%H%M%S")
    rootpath = '/g/data/hj10/cpol_level_1b/v2019/'
    ppifile = os.path.join(rootpath, f"ppi/{year}/{date_str}/twp10cpolppi.b1.{date_str}.{time_str}.nc")
    gridfile = os.path.join(rootpath, 
                            f"gridded/grid_150km_2500m/{year}/{date_str}/twp10cpolgrid150.b2.{date_str}.{time_str}.nc")
    if not os.path.exists(ppifile):
        return None
    if os.path.exists(gridfile):
        return None
    return ppifile

In [19]:
drange = pd.date_range('20090101', "20100101", freq='10Min')

In [20]:
bag = db.from_sequence(drange).map(get_file)
with ProgressBar():
    rslt = bag.compute()
    
rslt = [r for r in rslt if r is not None]

[########################################] | 100% Completed |  7.7s


In [21]:
rslt

['/g/data/hj10/cpol_level_1b/v2019/ppi/2009/20091201/twp10cpolppi.b1.20091201.231000.nc']

In [5]:
with open('missing_grids.json', 'w+') as fid:
    json.dump(rslt, fid)

In [6]:
total_flist = rslt

In [9]:
def grid_missing(infile):
    outpath = "/g/data/hj10/cpol_level_1b/v2019/gridded"    

    with warnings.catch_warnings():
        warnings.simplefilter('ignore')        

        try:
            grids.radar_gridding(infile, outpath)
        except KeyError:
            pass
        except Exception:
            traceback.print_exc()
    
    return None

In [10]:
def chunks(l, n):
    """
    Yield successive n-sized chunks from l.
    From http://stackoverflow.com/a/312464
    """
    for i in range(0, len(l), n):
        yield l[i:i + n]

In [25]:
grid_missing('/g/data/hj10/cpol_level_1b/v2019/ppi/2017/20170427/twp10cpolppi.b1.20170427.074000.nc')

twp10cpolppi.b1.20170427.074000.nc processed in 11.77.


In [22]:
# for flist in chunks(total_flist, 16):
#     bag = db.from_sequence(flist).map(grid_missing)
#     with ProgressBar():
#         _ = bag.compute()